# Rutinas para inferencia de institución y frecuencias de transmisión de datos de EMA, humedad del suelo -10cm

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

In [24]:
import pandas as pd
import numpy as np
import os
import re

In [4]:
# Se muestra el caso con mayor cantidad de frecuencias encontradas según slicing de [-10:] | Se cambia a [-100:] 10/2023
def analizar_frecuencias(carpeta, chunk_size=50000):
    archivos = os.listdir(carpeta)
    resultados = []

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
        
            periodo = []
            
            # Se itera sobre todos los chunk
            for chunk in reader:
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                sttn_val = chunk['Station'].values[0]
                
                try:
                    # Obtener frecuencia
                    periodos = pd.infer_freq(chunk['Fecha'][-200:])
                    periodo.append(periodos)
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
            
            # Agregar frecuencia al DataFrame
            if periodo:
                # Si se dedesa evaluar el comportamiento, se puede imprimir el siguiente statement
                #print(f'La frecuencia de los datos en el archivo {archivo} es {periodo[0]}')
                resultados.append({'Station': sttn_val, 'FreqInf': periodo[-1]})
    
    # Crear DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

In [6]:
# Uso función 
carpeta = 'RawUnmodified_HRS10'
df_resultados = analizar_frecuencias(carpeta) 
# Se copia en un bloc de notas y se guarda como un .csv

Error al inferir la frecuencia en el archivo Estacion_0024015110.csv: Need at least 3 dates to infer frequency
Error al inferir la frecuencia en el archivo Estacion_0026015010.csv: Need at least 3 dates to infer frequency
Error al inferir la frecuencia en el archivo Estacion_0026015040.csv: Need at least 3 dates to infer frequency
Error al inferir la frecuencia en el archivo Estacion_0026055120.csv: Need at least 3 dates to infer frequency
Error al inferir la frecuencia en el archivo Estacion_0035017020.csv: Need at least 3 dates to infer frequency
Error al inferir la frecuencia en el archivo Estacion_0046015030.csv: Need at least 3 dates to infer frequency
Error al inferir la frecuencia en el archivo Estacion_0054017040.csv: Need at least 3 dates to infer frequency


In [7]:
df_resultados.to_csv('freqs_HRS-10cm.csv')

In [7]:
# Ruta del directorio con los archivos y del archivo con las columnas lat y long
dir_path = 'RawUnmodified_HR/'
file_path = 'stations_all.csv'

# Cargamos el archivo csv que contiene las columnas 'lat' y 'long'
df = pd.read_csv(file_path)

# Inicializamos la lista donde almacenaremos los valores de lat y long
instituc_values = []
codigosests = []

# Lista que indica los nombres de los archivos no encontrados
missing_files = []

# Iteramos sobre todos los archivos en el directorio
for filename in os.listdir(dir_path):
    # Usamos regex para encontrar un número en el nombre del archivo
    match = re.search(r'\d+', filename)
    if match is not None:
        # Extraemos el número del nombre del archivo
        number = int(match.group())
        
        # Buscamos las filas que coinciden con este número
        matching_rows = df[df['Station'] == number]
        
        if matching_rows.empty:
            # Si no se encuentra ninguna fila correspondiente, se añade 
            #el nombre del archivo a la lista missing_files
            missing_files.append(filename)
        else:
            # Se extraen las instituciones a cargo
            for _, row in matching_rows.iterrows():
                instituc = row['project']
                cod = row['Station']
            
                # Se añaden los valores a la lista
                instituc_values.append(instituc)
                codigosests.append(cod)

checking = pd.DataFrame({'Instituc':instituc_values, 'Station':codigosests})

checking.head(2)

,Instituc,Station
0,CENICAFE,11025501
1,IDEAM,11030010


In [8]:
# Combina ambos DataFrame para tener frecuencias e institucion
frecinst100b = pd.merge(df_resultados, checking)
frecinst100b.head(2)
frecinst100b.to_csv('frecinst100b.csv')

Posteriormente, se hace una revisión manual de lo obtenido en el archivo 'frecinst100b.csv'. Se observan varias estaciones cuya frecuencia no pudo ser inferida que poseen comportamientos 'híbridos' a lo largo de sus transmisiones, por lo cual, se diligencia columna de observaciones, dejando en el registro la predominante.
Se exporta este nuevo archivo como 'frecinst100b_manual.csv' que es leído a continuación:

In [ ]:
## La siguiente, para comparar los dos resultados de [-10:] a [-100:] y 'FreqInf': periodo[0] a periodo[-1]
# Resultado [-100:]
freq100man = pd.read_csv('frecinst100b_manual.csv', sep=';')
# Resultado de inferencia de [-10:]:
freq10man = pd.read_csv('freq10binst_manualcorrect.csv')

In [ ]:
# Asumiendo que ambos DataFrames tienen una columna 'Station' en común
merged_df = pd.merge(freq100man, freq10man, on='Station')

# Crear una nueva columna 'diff' que sea True si 'FreqInf' es diferente de 'freq'
merged_df['diff'] = merged_df['FreqInf'] != merged_df['freq']

print(merged_df[merged_df['diff']])

# Filtrar las filas donde 'diff' es True
diff_stations = merged_df[merged_df['diff']]

# Seleccionar solo la columna 'Station' y obtener valores únicos
unique_stations = diff_stations['Station'].unique()

# Mostrar la lista de estaciones con diferencias
print(unique_stations.shape)

In [ ]:
merged_df['diff'] = (merged_df['FreqInf'] != merged_df['freq']) & pd.notna(merged_df['FreqInf'])

# Filtrar las filas donde 'diff' es True
diff_rows = merged_df[merged_df['diff']]

# Seleccionar solo las columnas 'FreqInf' y 'freq'
diff_values = diff_rows[['Station','FreqInf', 'freq','Observ']]

# Mostrar las diferencias
print(diff_values.shape)
print(diff_values)

In [12]:
CoordsHRS10 = pd.read_excel('EMA_HRS-10cm_coord.xlsx')
CoordsHRS10

,Unnamed: 0,Station,lat,long,entidad
0,0,11025501,5.888719,-76.145167,CENICAFE
1,1,11105020,7.154333,-76.977194,IDEAM
2,2,11115501,6.748275,-76.031556,CENICAFE
3,3,11135030,8.036722,-77.087861,IDEAM
4,4,12015100,7.940750,-76.696167,IDEAM
...,...,...,...,...,...
207,207,54017040,5.265111,-76.562750,IDEAM
208,208,54035501,4.726389,-76.114469,CENICAFE
209,209,55015010,5.521083,-76.973167,IDEAM
210,210,57025020,2.962944,-78.174361,IDEAM


In [3]:
CoordsHRS10.rename(columns={'CODIGO':'Station','LATITUD':'lat','LONGITUD':'long', 'ENTIDAD':'entidad'}, inplace=True)

In [16]:
freqs = pd.read_csv('freqs_HRS-10cm.csv', sep=';')
freqs

,Unnamed: 0,Station,FreqInf
0,0,11025501,5min
1,1,11105020,NaN
2,2,11115501,5min
3,3,11135030,NaN
4,4,12015100,NaN
...,...,...,...
200,200,53115502,5min
201,201,54035501,5min
202,202,55015010,NaN
203,203,57025020,NaN


In [20]:
partialdf_HRS10 = pd.merge(CoordsHRS10, freqs[['Station','FreqInf']], on='Station', how='left')
partialdf_HRS10

,Unnamed: 0,Station,lat,long,entidad,FreqInf
0,0,11025501,5.888719,-76.145167,CENICAFE,5min
1,1,11105020,7.154333,-76.977194,IDEAM,NaN
2,2,11115501,6.748275,-76.031556,CENICAFE,5min
3,3,11135030,8.036722,-77.087861,IDEAM,NaN
4,4,12015100,7.940750,-76.696167,IDEAM,NaN
...,...,...,...,...,...,...
207,207,54017040,5.265111,-76.562750,IDEAM,NaN
208,208,54035501,4.726389,-76.114469,CENICAFE,5min
209,209,55015010,5.521083,-76.973167,IDEAM,NaN
210,210,57025020,2.962944,-78.174361,IDEAM,NaN


In [22]:
partialdf_HRS10.to_csv('EMAHRS10_LatLonEntFreq.csv', index=False, encoding='latin-1')